# Reduce Pollution

This notebook outlines the general workflow for the data within the [Reduce Pollution](https://oceancentral.org/track/reduce-pollution) page of the Ocean Central website.

# Plastic Pollution

## Figure 1

<p align="center">
  <img src="Figs/reduce_pollution_plastic_1.png" style="width:50%;">
</p>

Data were obtained from OECD (2022) – processed by Our World in Data and can be found [here](https://ourworldindata.org/grapher/plastic-waste-accumulated-in-oceans).

## Figure 2

<p align="center">
  <img src="Figs/reduce_pollution_plastic_2.png" style="width:50%;">
</p>

Data were obtained from Meijer et al. (2021) – processed by Our World in Data and can be found [here](https://ourworldindata.org/plastic-pollution#explore-data-on-plastic-pollution).

## Figure 3

<p align="center">
  <img src="Figs/reduce_pollution_plastic_3.png" style="width:50%;">
</p>

Data were obtained from OECD (2022) – processed by Our World in Data and can be found [here](https://ourworldindata.org/grapher/plastic-production-by-sector).

## Figures 4 and 10

<p align="center">
  <img src="Figs/reduce_pollution_plastic_4.png" style="width:50%;">
</p>

<p align="center">
  <img src="Figs/reduce_pollution_plastic_10.png" style="width:50%;">
</p>

Data were obtained from United Nations Comtrade Database (2023), United Nations Comtrade Database (2025) – with minor processing by Our World in Data and can be found [here](https://ourworldindata.org/plastic-pollution#explore-data-on-plastic-pollution).

## Figures 5 and 11

<p align="center">
  <img src="Figs/reduce_pollution_plastic_5.png" style="width:50%;">
</p>

<p align="center">
  <img src="Figs/reduce_pollution_plastic_11.png" style="width:50%;">
</p>

Data were obtained from United Nations Comtrade Database (2023), United Nations Comtrade Database (2025) – with minor processing by Our World in Data and can be found [here](https://ourworldindata.org/plastic-pollution#explore-data-on-plastic-pollution).

## Figure 6

<p align="center">
  <img src="Figs/reduce_pollution_plastic_6.png" style="width:50%;">
</p>

Data were obtained from OECD (2022) – processed by Our World in Data and can be found [here](https://ourworldindata.org/plastic-waste-trade).

## Figure 7

<p align="center">
  <img src="Figs/reduce_pollution_plastic_7.png" style="width:50%;">
</p>

Data were obtained from OECD (2022) – processed by Our World in Data and can be found [here](https://ourworldindata.org/how-much-plastic-waste-ends-up-in-the-ocean).

## Figure 8

<p align="center">
  <img src="Figs/reduce_pollution_plastic_8.png" style="width:50%;">
</p>

Data were obtained from Meijer et al. (2021). More than 1000 rivers account for 80% of global riverine plastic emissions into the ocean. Science Advances. – processed by Our World in Data and can be found [here](https://ourworldindata.org/grapher/plastics-top-rivers?overlay=sources).

## Figure 9

<p align="center">
  <img src="Figs/reduce_pollution_plastic_9.png" style="width:50%;">
</p>

Data were obtained from Plastics in Great Pacific Garbage Patch (Lebreton et al. 2022) – processed by Our World in Data and can be found [here](https://ourworldindata.org/plastic-great-pacific-garbage).

## Figure 12

<p align="center">
  <img src="Figs/reduce_pollution_plastic_12.png" style="width:50%;">
</p>

Data were obtained from [Lebreton, Laurent; Reisser, Julia (2018). Supplementary data for 'River plastic emissions to the world's oceans'. figshare. Dataset. https://doi.org/10.6084/m9.figshare.4725541.v6](https://figshare.com/articles/dataset/River_plastic_emissions_to_the_world_s_oceans/4725541).

In [ ]:
#!/usr/bin/env python3
"""
Subset The Ocean Cleanup / Figshare 'River plastic emissions to the world's oceans'
dataset to the top 1000 river emitters and save as GeoJSON.

Assumes the shapefile components:
  PlasticRiverInputs.shp, .shx, .dbf, .prj, .qpj
are in the working directory.
"""

from pathlib import Path
import geopandas as gpd

# ---- CONFIG ----
INPUT_SHP = Path("../Data/PlasticRiverInputs/PlasticRiverInputs.shp")   
OUTPUT_GEOJSON = Path("Fig_12_plastic.geojson")
EMISSION_COL = "i_mid"  # mid estimate of plastic input

def main():
    # 1. Load shapefile
    if not INPUT_SHP.exists():
        raise FileNotFoundError(f"Shapefile not found: {INPUT_SHP}")

    gdf = gpd.read_file(INPUT_SHP)
    print(f"Loaded {len(gdf)} river points")
    print("Columns:", list(gdf.columns))

    # 2. Ensure emission column exists and is numeric
    if EMISSION_COL not in gdf.columns:
        raise ValueError(
            f"Emission column '{EMISSION_COL}' not found. "
            f"Available columns: {list(gdf.columns)}"
        )

    gdf[EMISSION_COL] = gdf[EMISSION_COL].astype(float)

    # 3. Drop rows with missing emission values (just in case)
    gdf = gdf[gdf[EMISSION_COL].notna()]

    # 4. Sort by emission and take top 1000
    gdf_sorted = gdf.sort_values(EMISSION_COL, ascending=False)
    top_n = min(1000, len(gdf_sorted))
    top1000 = gdf_sorted.head(top_n).reset_index(drop=True)

    print(f"Selected top {top_n} rivers by '{EMISSION_COL}'")

    # 5. Make sure CRS is WGS84 (it already is in your file, EPSG:4326)
    if gdf.crs is None:
        # If for some reason CRS is missing, set it explicitly
        top1000.set_crs(epsg=4326, inplace=True)
    else:
        top1000 = top1000.to_crs(epsg=4326)

    # 6. Save to GeoJSON
    top1000.to_file(OUTPUT_GEOJSON, driver="GeoJSON")
    print(f"Saved GeoJSON to: {OUTPUT_GEOJSON.resolve()}")

if __name__ == "__main__":
    main()
